# Соревнование по сентимент-анализу
#### Стародубец Андрей

Импортируем все необходимые библиотеки

In [1]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

import nltk
import re
import string

Данные для обучения и решения:

In [2]:
train = pd.read_csv('products_sentiment_train.tsv', sep='\t', header=None, names=['text', 'target'])
test = pd.read_csv('products_sentiment_test.tsv', sep='\t')

In [3]:
train.head()

,text,target
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


Для дальнейшей векторизации превращаем данные в списки

In [4]:
texts = train["text"].tolist()
labels = train["target"].tolist()

Построим предсказания на основе TfidfVectorizer (параметры были подобраны методом проб и ошибок) и LinearSVC с дефолтными параметрами

In [5]:
clf = Pipeline([('vectorizer',TfidfVectorizer(ngram_range = (1,3), min_df = 2)), ('classifier', LinearSVC())])
cv = cross_val_score(clf, texts, labels, cv = 5).mean()
print (cv)  

0.7824894436840231


Результаты удовлетворительные. Предскажем класс отзывов:

In [6]:
clf = Pipeline([('vectorizer',TfidfVectorizer(ngram_range = (1,3), min_df = 2)), ('classifier', LinearSVC())])
clf.fit(texts, labels)
test['y'] = clf.predict(test["text"])

И запишем в файл для проверки:

In [7]:
test[['Id','y']].to_csv('products_sentiment_submission.csv', index = None)